In [1]:
from collections import defaultdict
from django.db.models.signals import *
import pandas as pd
from itertools import chain
from tqdm.notebook import tqdm
import datetime
import calendar

In [2]:
def normalize_month(month, year):
    return month + 12*(year - 2020)

In [3]:
normalize_month(12, 2023)

48

In [4]:
# def normalize_day(day, month):
#     return day + 31*(month - 1)

In [5]:
# normalize_day(31,12)

In [6]:
def queryset_to_dict(result_list):
    permissoesUsuarios = {}
    for a in result_list:
        try:
            if not permissoesUsuarios.get(a.user.get_full_name()):
                permissoesUsuarios[a.user.get_full_name()] = {}
            if isinstance(permissoesUsuarios.get(a.user.get_full_name()).get(normalize_month(a.history_date.month, a.history_date.year)),list):
                permissoesUsuarios[a.user.get_full_name()][normalize_month(a.history_date.month, a.history_date.year)].append(a.permissions.name)
            elif permissoesUsuarios.get(a.user.get_full_name()).get(normalize_month(a.history_date.month, a.history_date.year)):
                multi_perm_per_month= permissoesUsuarios.get(a.user.get_full_name()).get(normalize_month(a.history_date.month, a.history_date.year))
                permissoesUsuarios[a.user.get_full_name()][normalize_month(a.history_date.month, a.history_date.year)] = [multi_perm_per_month,a.permissions.name]
            else:
                permissoesUsuarios[a.user.get_full_name()][normalize_month(a.history_date.month, a.history_date.year)] = a.permissions.name
        except:
            pass
    return permissoesUsuarios

In [7]:
def cond_ativo_inativo(permissoesUsuarios,mes_referencia):
    usuariosAtivos = []
    usuariosInativos = []
    
    for nome, mes in permissoesUsuarios.items():
        if mes == {}:
            continue
        meses = sorted(list(mes.keys()))
        item = max(list(mes.keys()))
        cargo_atual = mes[item]
        
        if len(meses) > 1:
            if mes[meses[-2]] != 'Inativo':
                cargo_antes_de_inativo = True
            else:
                cargo_antes_de_inativo = False
        else:
            cargo_antes_de_inativo = False
        
    # Se em algum momento do mesmo mês o usuário foi diferente de Inativo ele entra como ativo
        if isinstance(cargo_atual,list):
            if any(cargo for cargo in cargo_atual if cargo !='Inativo') and not any(u for u in usuariosAtivos if nome == u):
                usuariosAtivos.append(nome)     
    # se o cargo atual for diferente de inativo e não for uma lista(o que configura multiplas permissões no mesmo mês) ele é ativo
        elif cargo_atual != 'Inativo' and not isinstance(cargo_atual,list):
            usuariosAtivos.append(nome)
    # Se no mes anterior ao ultimo ele foi diferente de inativo e no ultimo mes foi Inativo e
    # o mes do cargo_atual for igual ao ultimo mes de referencia(ultimo mês pra coleta) e o nome não estiver nos usuariosAtivos
    # ele é ativo, pois se nesse mês ele foi colocado como Inativo 
    # quer dizer que em algum momento do mês de referencia ele foi ativo
        elif cargo_antes_de_inativo and cargo_atual == 'Inativo' and item == mes_referencia and not any(u for u in usuariosAtivos if nome == u):
            usuariosAtivos.append(nome)       
    # se o cargo atual contem a palavra Inativo e não está nos usuariosAtivos então ele é Inativo
        elif 'Inativo' in cargo_atual and nome not in usuariosAtivos:
            usuariosInativos.append(nome)
        else:
            print('Corra pras montanhas')
    
    return usuariosAtivos,usuariosInativos

In [8]:
def permissoes_history_user(result_list,mes_ano):
    temp_ativo,temp_inativo,usuariosAtivos,usuariosInativos=[],[],[],[]
    
    permissoesUsuarios = {}
    
    mes_referencia = normalize_month(mes_ano.month, mes_ano.year)
    companies = list(result_list.values_list('company__name',flat=True).order_by('company__name').distinct('company__name'))
    
    # condições para tratar dos casos de agrupamento de unidades
    if len(companies) == 1:
        permissoesUsuarios = queryset_to_dict(result_list)
        
        usuariosAtivos,usuariosInativos=cond_ativo_inativo(permissoesUsuarios,mes_referencia)
    
    # se há mais de uma unidade quer dizer q foi feito um agrupamentos por unidade e temos que
    # iterar sobre cada unidade separadamente, pra no fim retornar a lista de usuários por causa de um erro de o usuário
    # estar como inativo numa e ativo na outra
    elif len(companies) > 1:
        
        for company in companies:
            result_list_filtered = result_list.filter(company__name=company)
            permissoesUsuarios = queryset_to_dict(result_list_filtered)

            temp_ativo=cond_ativo_inativo(permissoesUsuarios,mes_referencia)[0] + temp_ativo
            temp_inativo=cond_ativo_inativo(permissoesUsuarios,mes_referencia)[1] + temp_inativo
            
        try:
            usuariosInativos = list(set([x for x in temp_inativo if x not in temp_ativo]))
            usuariosAtivos = list(set(temp_ativo))
        except:
            print(usuariosAtivos,usuariosInativos)
    return usuariosAtivos,usuariosInativos

In [9]:
companies=['Modera - Consórcio ME2',
  'Afirma Engenharia Viária',
  'Pavicolet (Amapá)',
  'Pavicolet (Amazonas)',
  'Entrevias',
  'LCM - Gaspar',
  'Minasul',
  'Sinalizavia (Municípios)',
  'Sinaliza Via (Rodovias)',
  'Construtora Castilho']
len(companies),companies

(10,
 ['Modera - Consórcio ME2',
  'Afirma Engenharia Viária',
  'Pavicolet (Amapá)',
  'Pavicolet (Amazonas)',
  'Entrevias',
  'LCM - Gaspar',
  'Minasul',
  'Sinalizavia (Municípios)',
  'Sinaliza Via (Rodovias)',
  'Construtora Castilho'])

In [19]:
companies=list(Company.objects.filter(name__icontains='Construtora Castilho').values_list('cnpj',flat=True).distinct())

In [20]:
companies

['92.779.503/0001-25']

In [10]:
for a in companies:
    company=Company.objects.get(name=a)
    print(company.name,company.cnpj)

Modera - Consórcio ME2 28.256.567/0001-42
Afirma Engenharia Viária 05.205.684/0001-81
Pavicolet (Amapá) 46.791.721/0001-68
Pavicolet (Amazonas) 46.791.721/0001-68
Entrevias 26.664.057/0001-89
LCM - Gaspar 19.758.842/0001-35
Minasul 37.456.439/0001-33
Sinalizavia (Municípios) 22.221.801/0001-10
Sinaliza Via (Rodovias) 22.221.801/0001-10
Construtora Castilho 92.779.503/0001-25


In [11]:
# companies=[a for a in Company.objects.filter(name__startswith='LCM').values_list('name',flat=True)]
# companies

In [13]:
# Mês e ano para coleta dos usuários
mes=3
ano=2024
dia = calendar.monthrange(ano, mes)[1]
dia,mes,ano

(31, 3, 2024)

In [14]:
# DATA sempre referente ao mês que se quer coletar nesse formato history_date__lte='2024-01-31 03:00:00'
# é utilizado 3 horas da manha pois é a pra fechar com o horario de meio noite do servidor
data = datetime.datetime(ano,mes,dia,3,0,0)
data

datetime.datetime(2024, 3, 31, 3, 0)

In [15]:
pd.DataFrame().to_excel('Usuários Ativos Financeiro.xlsx')

In [16]:
for company in tqdm(companies):
    planilha_name=company
    if company in ['Modera - Consórcio ME2']:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__in=['Modera - Consórcio ME2'],
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    elif company in ['Afirma Engenharia Viária']:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__in=['Afirma Engenharia Viária'],
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    elif company in ['Pavicolet (Amapá)','Pavicolet (Amazonas)']:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__in=['Pavicolet (Amapá)','Pavicolet (Amazonas)'],
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
        planilha_name='Pavicolet'
    elif company in ['LCM - Gaspar']:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__in=['LCM - Gaspar'],
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    elif company in ['Minasul']:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__in=['Minasul'],
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    elif company in ['Sinalizavia (Municípios)','Sinaliza Via (Rodovias)']:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__in=['Sinalizavia (Municípios)','Sinaliza Via (Rodovias)'],
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
        planilha_name='Sinalizavia'
    elif company in ['Construtora Castilho']:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__in=['Construtora Castilho'],
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    elif company in ['Entrevias']:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            Q(user__email__icontains='afirma')|Q(user__email__icontains='ajr')|Q(user__username__icontains='ajr'),
            company__name__icontains=company,
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    else:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name=company,
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    
    users_permissions = permissoes_history_user(historicoUsuarios,data)[0] if len(permissoes_history_user(historicoUsuarios,data)[0]) > 0 else 0
#     cnpj=Company.objects.get(name=company).cnpj.replace('.', '').replace('/', '').replace('-', '')
    if users_permissions != 0:
        df2 = pd.DataFrame(data=users_permissions,columns=['Usuários ativos'])
        with pd.ExcelWriter('Usuários Ativos Financeiro.xlsx', mode='a',if_sheet_exists='replace') as writer:
            df2.to_excel(writer,sheet_name=planilha_name,na_rep="-")

  0%|          | 0/5 [00:00<?, ?it/s]

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField HistoricalUserInCompany.history_date received a naive datetime (2024-03-31 03:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [85]:
historicoUsuarios=UserInCompany.history.model.objects.filter(
        company__name__in=['Pavicolet (Amapá)','Pavicolet (Amazonas)'],
        history_date__lte=data
    ).exclude(
        user__email__icontains='kartado'
    ).exclude(
        user__email__icontains='roadlabs'
    ).exclude(
        user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
    ).exclude(
        user__email__icontains='hermes'
    ).order_by( 'history_date')

In [86]:
users_permissions1 = permissoes_history_user(historicoUsuarios,data)[0] if len(permissoes_history_user(historicoUsuarios,data)[0]) > 0 else 0

In [87]:
permissoes_history_user(historicoUsuarios,data),len(users_permissions1)

((['Edielsom Campos',
   'HENRIQUE MOUSSA',
   'Jobson Lima',
   'Welington Bortolini',
   'JOÃO ECCEL',
   'Nelson Roriz',
   'Cleiton Marques',
   'WELINGTON DIEGO COSTA',
   'RODRIGO SOUZA',
   'Leonardo Meireles Martins',
   'ELIZANIO VIEIRA',
   'Eliton costa',
   'Edmilson Peixoto',
   'Alexandre Assef',
   'Vilmar Júnior',
   'Diego de Jesus Amoras',
   'Flávio Tavares',
   'Pedro Barbosa',
   'Ruan Carlos Souza da Silva',
   'Álvaro Aurélio',
   'Isenildo Torres',
   'Hugo Isslle',
   'Mailso Dias',
   'Jonas Silva',
   'Mateus Tavares',
   'Wellington Fagundes',
   'Carlos Porto',
   'Edgar Silva',
   'Neandro Braga',
   'Jonnattan Gustavo Monteiro de Matos',
   'Wal-liano Deka',
   'Adilson Rodrigues',
   'Wembley Santos',
   'Cleimilton Melo',
   'Marcus Vinicius C Abreu',
   'Tassio Janio'],
  ['Joao Oliveira',
   'Saulo Daniel',
   'fredson venque',
   'Marcelo Martins',
   'Cristiano Oliveira',
   'Aderbal Relvas',
   'Abrão Baloeiro Dutra',
   'Antônio Barbosa',
   'Anto

In [40]:
users_permissions2 = permissoes_history_user(historicoUsuarios,data)[0] if len(permissoes_history_user(historicoUsuarios,data)[0]) > 0 else 0

In [43]:
len(users_permissions2),users_permissions2

(15,
 ['Heverton Canova',
  'João Fachini',
  'Sergio Deves',
  'Celestino Cuchi',
  'Celso dos Santos',
  'Cleberson Rocha',
  'Delbo Rodrigues',
  'Evandro Pereira',
  'Flávio Soares',
  'Jhonatan dos Santos',
  'José da Costa',
  'Luiz Antônio',
  'Rafael Carvalho',
  'Ricardo Guilherme',
  'Paulo Marchel'])

In [53]:
len(set(users_permissions1+users_permissions2))

15

In [47]:
len(new_lista),new_lista

(15,
 ['Heverton Canova',
  'João Fachini',
  'Cleberson Rocha',
  'Delbo Rodrigues',
  'Evandro Pereira',
  'Jhonatan dos Santos',
  'José da Costa',
  'Ricardo Guilherme',
  'Luiz Antônio',
  'Celso dos Santos',
  'Flávio Soares',
  'Paulo Marchel',
  'Celestino Cuchi',
  'Sergio Deves',
  'Rafael Carvalho'])

In [45]:
historicoUsuarios=UserInCompany.history.model.objects.filter(
            Q(user__email__icontains='afirma')|Q(user__email__icontains='ajr')|Q(user__username__icontains='ajr'),
            company__name__icontains="Entrevias",
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')

In [46]:
users_permissions = permissoes_history_user(historicoUsuarios,data)[0]
users_permissions

['Isabelly AJR',
 'Rogério Buchala',
 'Cassio Borges',
 'Luciano Teixeira',
 'Osmar Duprat',
 'Marcio Celestino']

In [88]:
def queryset_to_dict(result_list):
    permissoesUsuarios = {}
    for a in result_list:
        try:
            if not permissoesUsuarios.get(a.user.get_full_name()):
                permissoesUsuarios[a.user.get_full_name()] = {}
            if isinstance(permissoesUsuarios.get(a.user.get_full_name()).get(normalize_month(a.history_date.month, a.history_date.year)),list):
                permissoesUsuarios[a.user.get_full_name()][normalize_month(a.history_date.month, a.history_date.year)].append(a.permissions.name)
            elif permissoesUsuarios.get(a.user.get_full_name()).get(normalize_month(a.history_date.month, a.history_date.year)):
                multi_perm_per_month= permissoesUsuarios.get(a.user.get_full_name()).get(normalize_month(a.history_date.month, a.history_date.year))
                permissoesUsuarios[a.user.get_full_name()][normalize_month(a.history_date.month, a.history_date.year)] = [multi_perm_per_month,a.permissions.name]
            else:
                permissoesUsuarios[a.user.get_full_name()][normalize_month(a.history_date.month, a.history_date.year)] = a.permissions.name
        except:
            pass
    return permissoesUsuarios

In [104]:
def cond_ativo_inativo(permissoesUsuarios,mes_referencia):
    usuariosAtivos = []
    usuariosInativos = []
    
    for nome, mes in permissoesUsuarios.items():
        if mes == {}:
            continue
        meses = sorted(list(mes.keys()))
        item = max(list(mes.keys()))
        cargo_atual = mes[item]
        
        if len(meses) > 1:
            if mes[meses[-2]] != 'Inativo':
                cargo_antes_de_inativo = True
            else:
                cargo_antes_de_inativo = False
        else:
            cargo_antes_de_inativo = False
        
    # Se em algum momento do mesmo mês o usuário foi diferente de Inativo ele entra como ativo
        if isinstance(cargo_atual,list):
            if any(cargo for cargo in cargo_atual if cargo !='Inativo') and not any(u for u in usuariosAtivos if nome == u):
                usuariosAtivos.append(nome)     
    # se o cargo atual for diferente de inativo e não for uma lista(o que configura multiplas permissões no mesmo mês) ele é ativo
        elif cargo_atual != 'Inativo' and not isinstance(cargo_atual,list):
            usuariosAtivos.append(nome)
    # Se no mes anterior ao ultimo ele foi diferente de inativo e no ultimo mes foi Inativo e
    # o mes do cargo_atual for igual ao ultimo mes de referencia(ultimo mês pra coleta) e o nome não estiver nos usuariosAtivos
    # ele é ativo, pois se nesse mês ele foi colocado como Inativo 
    # quer dizer que em algum momento do mês de referencia ele foi ativo
        elif cargo_antes_de_inativo and cargo_atual == 'Inativo' and item == mes_referencia and not any(u for u in usuariosAtivos if nome == u):
            usuariosAtivos.append(nome)       
    # se o cargo atual contem a palavra Inativo e não está nos usuariosAtivos então ele é Inativo
        elif 'Inativo' in cargo_atual and nome not in usuariosAtivos:
            usuariosInativos.append(nome)
        else:
            print('Corra pras montanhas')
    
    return usuariosAtivos,usuariosInativos

In [112]:
def permissoes_history_user(result_list,mes_ano):
    temp_ativo,temp_inativo,usuariosAtivos,usuariosInativos=[],[],[],[]
    
    permissoesUsuarios = {}
    
    mes_referencia = normalize_month(mes_ano.month, mes_ano.year)
    companies = list(result_list.values_list('company__name',flat=True).order_by('company__name').distinct('company__name'))
    
    # condições para tratar dos casos de agrupamento de unidades
    if len(companies) == 1:
        permissoesUsuarios = queryset_to_dict(result_list)
        
        usuariosAtivos,usuariosInativos=cond_ativo_inativo(permissoesUsuarios,mes_referencia)
    
    # se há mais de uma unidade quer dizer q foi feito um agrupamentos por unidade e temos que
    # iterar sobre cada unidade separadamente, pra no fim retornar a lista de usuários por causa de um erro de o usuário
    # estar como inativo numa e ativo na outra
    elif len(companies) > 1:
        
        for company in companies:
            result_list_filtered = result_list.filter(company__name=company)
            permissoesUsuarios = queryset_to_dict(result_list_filtered)

            temp_ativo=cond_ativo_inativo(permissoesUsuarios,mes_referencia)[0] + temp_ativo
            temp_inativo=cond_ativo_inativo(permissoesUsuarios,mes_referencia)[1] + temp_inativo
            
        try:
            usuariosInativos = list(set([x for x in temp_inativo if x not in temp_ativo]))
            usuariosAtivos = list(set(temp_ativo))
        except:
            print(usuariosAtivos,usuariosInativos)
    return usuariosAtivos,usuariosInativos

In [116]:
historicoUsuarios=UserInCompany.history.model.objects.filter(
        company__name__in=['Sinalizavia (Municípios)','Sinaliza Via (Rodovias)'],
        history_date__lte=data
    ).exclude(
        user__email__icontains='kartado'
    ).exclude(
        user__email__icontains='roadlabs'
    ).exclude(
        user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
    ).exclude(
        user__email__icontains='hermes'
    ).order_by( 'history_date','company')

In [117]:
users_permissions1 = permissoes_history_user(historicoUsuarios,data)[0] if len(permissoes_history_user(historicoUsuarios,data)[0]) > 0 else 0

In [118]:
permissoes_history_user(historicoUsuarios,data),len(users_permissions1)

((['João Fachini',
   'Delbo Rodrigues',
   'Heverton Canova',
   'Celso dos Santos',
   'Celestino Cuchi',
   'Paulo Marchel',
   'Cleberson Rocha',
   'Flávio Soares',
   'Ricardo Guilherme',
   'José da Costa',
   'Jhonatan dos Santos',
   'Sergio Deves',
   'Luiz Antônio',
   'Rafael Carvalho',
   'Evandro Pereira'],
  ['Eduardo Rezende', 'Marcos Fachini', 'Alessandro Maia']),
 15)

In [58]:
def denormalize_month(normalized_value):
    year = 2020 + normalized_value // 12
    month = normalized_value % 12
    return month, year

# Exemplo de uso
normalized_value = normalize_month(2, 2024)
month, year = denormalize_month(normalized_value)

In [114]:
desnormalize_month(2,2024)

50